# Customer Churn Prediction - CRISP-DM Analysis

Dự án này thực hiện phân tích và dự đoán khách hàng rời bỏ (Churn) theo quy trình CRISP-DM gồm 6 giai đoạn:
1. Business Understanding
2. Data Understanding (EDA)
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment


In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Thêm thư mục src vào path để import modules
sys.path.append(os.path.abspath('..'))

from src.preprocessing import load_data, perform_eda, build_preprocessor
from src.modeling import train_and_evaluate

# Cấu hình hiển thị
try:
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Đã import các thư viện thành công!")


## 1. Business Understanding

**Mục tiêu:** Xác định khách hàng có nguy cơ rời bỏ (Churn) để doanh nghiệp có thể có chiến lược giữ chân khách hàng hiệu quả.

**Đầu ra:** 
- Nhãn dự đoán: Churn (1) hoặc No Churn (0)
- Xác suất churn để đánh giá mức độ rủi ro


## 2. Data Understanding (EDA)


In [ ]:
# Load dữ liệu
df = load_data()
print(f"\nKích thước dữ liệu: {df.shape}")
print(f"\nCác cột trong dataset:")
print(df.columns.tolist())


In [ ]:
# Xem thông tin cơ bản về dữ liệu
print("=== THÔNG TIN TỔNG QUAN ===")
print(f"\nSố dòng: {df.shape[0]}")
print(f"Số cột: {df.shape[1]}")
print(f"\nKiểu dữ liệu:")
print(df.dtypes)
print(f"\nThông tin tổng quan:")
df.info()


In [ ]:
# Xem 5 dòng đầu tiên
df.head()


In [ ]:
# Kiểm tra giá trị thiếu
print("=== GIÁ TRỊ THIẾU ===")
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Số lượng thiếu': missing_values,
    'Tỷ lệ (%)': missing_percent
})
missing_df = missing_df[missing_df['Số lượng thiếu'] > 0].sort_values('Số lượng thiếu', ascending=False)
if len(missing_df) > 0:
    print(missing_df)
else:
    print("Không có giá trị thiếu trong dataset!")


In [ ]:
# Thống kê mô tả cho các biến số
print("=== THỐNG KÊ MÔ TẢ ===")
df.describe()


In [ ]:
# Phân phối của biến target (Churn)
print("=== PHÂN PHỐI BIẾN TARGET (CHURN) ===")
churn_counts = df['Churn'].value_counts()
churn_percent = df['Churn'].value_counts(normalize=True) * 100

print(f"\nSố lượng:")
print(churn_counts)
print(f"\nTỷ lệ (%):")
print(churn_percent)

# Vẽ biểu đồ
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar plot
axes[0].bar(churn_counts.index.astype(str), churn_counts.values, color=['skyblue', 'salmon'])
axes[0].set_title('Phân phối Churn', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Churn')
axes[0].set_ylabel('Số lượng')
axes[0].set_xticklabels(['No Churn (0)', 'Churn (1)'])

# Pie chart
axes[1].pie(churn_counts.values, labels=['No Churn', 'Churn'], autopct='%1.1f%%', 
            colors=['skyblue', 'salmon'], startangle=90)
axes[1].set_title('Tỷ lệ Churn', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
# Phân tích tương quan giữa các biến số
print("=== MA TRẬN TƯƠNG QUAN ===")
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

# Vẽ heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Ma trận tương quan giữa các biến số', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Tương quan với Churn
if 'Churn' in corr_matrix.columns:
    churn_corr = corr_matrix['Churn'].sort_values(ascending=False)
    print("\nTương quan với Churn:")
    print(churn_corr)


In [ ]:
# Phân tích các biến phân loại
print("=== PHÂN TÍCH BIẾN PHÂN LOẠI ===")
categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if col != 'CustomerID':  # Bỏ qua CustomerID
        print(f"\n{col}:")
        print(df[col].value_counts())
        print(f"Tỷ lệ Churn theo {col}:")
        churn_by_cat = df.groupby(col)['Churn'].agg(['count', 'sum', 'mean'])
        churn_by_cat.columns = ['Tổng số', 'Số Churn', 'Tỷ lệ Churn']
        print(churn_by_cat)
        print("-" * 50)


In [ ]:
# Vẽ biểu đồ phân phối các biến số quan trọng
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Churn' in numeric_features:
    numeric_features.remove('Churn')
if 'CustomerID' in numeric_features:
    numeric_features.remove('CustomerID')

n_features = len(numeric_features)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes

for i, col in enumerate(numeric_features[:len(axes)]):
    axes[i].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[i].set_title(f'Phân phối {col}', fontweight='bold')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Tần suất')

# Ẩn các subplot không sử dụng
for i in range(len(numeric_features), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()


## 3. Data Preparation

Tiền xử lý dữ liệu bao gồm:
- Xử lý giá trị thiếu
- Chuẩn hóa các biến số
- Mã hóa các biến phân loại


In [ ]:
# Thực hiện EDA và xây dựng preprocessor
print("=== TIỀN XỬ LÝ DỮ LIỆU ===")
eda_stats = perform_eda(df)
print("\nĐã hoàn thành EDA!")

# Xây dựng preprocessor
preprocessor, X, y = build_preprocessor(df)
print(f"\nSố features sau preprocessing: {len(X.columns)}")
print(f"Features số: {list(X.select_dtypes(include=[np.number]).columns)}")
print(f"Features phân loại: {list(X.select_dtypes(include=['object']).columns)}")


In [ ]:
# Fit preprocessor
print("Đang fit preprocessor...")
preprocessor.fit(X)
print("Đã fit preprocessor thành công!")

# Xem số lượng features sau khi transform
X_transformed = preprocessor.transform(X)
print(f"\nSố features sau transform: {X_transformed.shape[1]}")


## 4. Modeling & Evaluation

Huấn luyện và đánh giá các mô hình:
- Logistic Regression
- Random Forest
- SVM (Support Vector Machine)

Đánh giá dựa trên: Accuracy, Precision, Recall, F1-Score, ROC-AUC


In [ ]:
# Huấn luyện và đánh giá các mô hình
print("=== HUẤN LUYỆN VÀ ĐÁNH GIÁ MÔ HÌNH ===\n")
results = train_and_evaluate()


In [ ]:
# Hiển thị kết quả đánh giá
print("\n=== KẾT QUẢ ĐÁNH GIÁ CÁC MÔ HÌNH ===")
results_df = pd.DataFrame(results).T
results_df = results_df.round(4)
print(results_df)

# Vẽ biểu đồ so sánh
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
for i, metric in enumerate(metrics):
    if i < len(axes):
        results_df[metric].plot(kind='bar', ax=axes[i], color='steelblue')
        axes[i].set_title(f'{metric.upper()}', fontweight='bold')
        axes[i].set_ylabel('Score')
        axes[i].set_xticklabels(results_df.index, rotation=45, ha='right')
        axes[i].grid(axis='y', alpha=0.3)

# Ẩn subplot không sử dụng
for i in range(len(metrics), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Xác định mô hình tốt nhất
best_model = results_df['f1'].idxmax()
best_score = results_df.loc[best_model, 'f1']

print(f"\n=== MÔ HÌNH TỐT NHẤT ===")
print(f"Mô hình: {best_model}")
print(f"F1-Score: {best_score:.4f}")
print(f"\nTất cả metrics của mô hình tốt nhất:")
print(results_df.loc[best_model])


## 5. Deployment

Mô hình đã được lưu và có thể sử dụng qua API FastAPI.

**Các endpoint:**
- `GET /eda`: Xem kết quả EDA
- `POST /train`: Huấn luyện lại mô hình
- `POST /predict`: Dự đoán churn cho dữ liệu mới
- `GET /status`: Kiểm tra trạng thái hệ thống

**Chạy API:**
```bash
uvicorn demo.app:app --reload
```

Sau đó truy cập: http://127.0.0.1:8000/docs để xem tài liệu API.
